In [1]:
from __future__ import annotations

import cProfile
import pstats
from asyncio import sleep

In [3]:
from py_research.db import (
    DataSource,
    TableMap,
    SubTableMap,
    SubMap,
    SelIndex,
    DataBase
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\lworm\AppData\Local\Temp\ipykernel_13480\178736428.py:1 in <module>                     │
│                                                                                                  │
│ ❱ 1 from py_research.db import (                                                                 │
│   2 │   DataSource,                                                                              │
│   3 │   TableMap,                                                                                │
│   4 │   SubTableMap,                                                                             │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\__init__.py:4 in <module>                    │
│                                                                                                  │
│    1 """Universal relational database interface for python."""                                   │
│    2                                                                                             │
│    3 from .conflicts import DataConflictError as DataConflictError                               │
│ ❱  4 from .databases import Array as Array                                                       │
│    5 from .databases import BackLink as BackLink                                                 │
│    6 from .databases import Data as Data                                                         │
│    7 from .databases import DataBase as DataBase                                                 │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:4507 in <module>                │
│                                                                                                  │
│   4504 │   _to: Link[RecT3] = Link(index=True)                                                   │
│   4505                                                                                           │
│   4506                                                                                           │
│ ❱ 4507 class IndexedRelation(Relation[RecT2, RecT3], Generic[RecT2, RecT3, KeyT]):               │
│   4508 │   """Automatically defined relation record type with index substitution."""             │
│   4509 │                                                                                         │
│   4510 │   _template = True                                                                      │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:4164 in __init_subclass__       │
│                                                                                                  │
│   4161 │   │   │   │   │   compare=True,                                                         │
│   4162 │   │   │   │   │   kw_only=True,                                                         │
│   4163 │   │   │   │   )                                                                         │
│ ❱ 4164 │   │   │   │   for name, a in cls._values.items()                                        │
│   4165 │   │   │   },                                                                            │
│   4166 │   │   │   **{                                                                           │
│   4167 │   │   │   │   name: Field(                                                              │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:4098 in _values                 │
│                                                            

In [3]:
from test_db_schema import Search, Project, Task, User, Membership, Organization

In [4]:
async def dummy_loader(name: str) -> dict:
    await sleep(0.2)
    return {
        'name': name,
        'age': 20
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": SubTableMap(
            target=Search.results,
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": SubTableMap(
                    target=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": SubTableMap(
                            target=Task.assignees,
                            push=User.name,
                            match_by=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": SubTableMap(
                    target=Project.members,
                    loader=dummy_loader,
                    push={User.name, User.age},
                    rel_map=TableMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
from datetime import date


data = {
    "resultCount": 3,
    "search": "test",
    "results": [
        {
            "project_name": "baking cake",
            "project_start": date(2020, 1, 1),
            "project_end": date(2020, 1, 4),
            "project_status": "done",
            "organization_name": "Bakery",
            "organization_address": "Main Street 1",
            "organization_city": "Bakerville",
            "tasks": [
                {
                    "task_name": "task1",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task2",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task3",
                    "task_assignees": ["Jane"],
                    "task_status": "done",
                },
            ],
            "members": [
                {"name": "John", "role": "baker", "age": 30},
                {"name": "John", "role": "manager", "age": 40},
            ],
        },
        {
            "project_name": "cleaning shoes",
            "project_start": date(2020, 1, 2),
            "project_end": date(2020, 1, 5),
            "project_status": "done",
            "organization_name": "Shoe Shop",
            "organization_address": "Main Street 2",
            "organization_city": "Shoetown",
            "tasks": [
                {
                    "task_name": "task4",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task5",
                    "task_assignees": ["Jane"],
                    "task_status": "todo",
                },
            ],
            "members": [
                {"name": "John", "role": "cleaner", "age": 25},
                {"name": "Jane", "role": "manager", "age": 35},
            ],
        },
        {
            "project_name": "fixing cars",
            "project_start": date(2020, 1, 3),
            "project_end": date(2020, 1, 6),
            "project_status": "started",
            "organization_name": "Car Shop",
            "organization_address": "Main Street 3",
            "organization_city": "Cartown",
            "tasks": [
                {
                    "task_name": "task6",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                }
            ],
            "members": [
                {"name": "John", "role": "mechanic", "age": 45},
                {"name": "Jane", "role": "manager", "age": 55},
                {"name": "Jack", "role": "manager", "age": 65},
            ],
        },
    ],
}

In [ ]:
with cProfile.Profile() as pr:
  db = DataBase()
  rec = await data_source.load([data], db=db)

  stats = pstats.Stats(pr)


Async-loading `Search`: 100%|██████████| 1/1 [00:00<00:00,  9.57it/s]
                                                                     
Async-loading `Project`:  33%|███▎      | 1/3 [06:13<12:27, 373.88s/it]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\lworm\AppData\Local\Temp\ipykernel_5916\3418091003.py:3 in <module>                     │
│                                                                                                  │
│   1 with cProfile.Profile() as pr:                                                               │
│   2   db = DataBase()                                                                            │
│ ❱ 3   rec = await data_source.load([data], db=db)                                                │
│   4                                                                                              │
│   5   stats = pstats.Stats(pr)                                                                   │
│   6                                                                                              │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:871 in load                   │
│                                                                                                  │
│   868 │   │   db = db if db is not None else DataBase()                                          │
│   869 │   │   in_data: InData = {((), ()): dat for dat in data}                                  │
│   870 │   │   rest_data: RestData = []                                                           │
│ ❱ 871 │   │   loaded = await _load_records(                                                      │
│   872 │   │   │   db, cast(TableMap[Record, TreeNode], self), in_data, rest_data                 │
│   873 │   │   )                                                                                  │
│   874                                                                                            │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:823 in _load_records          │
│                                                                                                  │
│   820 │   for rel_tgt, sub_data in lazy_data:                                                    │
│   821 │   │   if isinstance(rel_tgt, SubTableMap):                                               │
│   822 │   │   │   # Full record relation set                                                     │
│ ❱ 823 │   │   │   rec_set[rel_tgt.target] |= await _load_records(                                │
│   824 │   │   │   │   db,                                                                        │
│   825 │   │   │   │   rel_tgt,                                                                   │
│   826 │   │   │   │   cast(InData, sub_data),                                                    │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:782 in _load_records          │
│                                                                                                  │
│   779 │   records: dict[Hashable, Record | Hashable] = {}                                        │
│   780 │   rest_records: InData = {}                                                              │
│   781 │                                                                                          │
│ ❱ 782 │   async for ((parent_idx, path_idx), data), (rec, rel) in loaded:                        │
│   783 │   │   if rec is None:                                                                    │
│   784 │   │   │   rest_records[(parent_idx, path_idx)] = data                                    │
│   785 │   │   │   continue                                                                       │
╰────────────────────────────────────────────────────────────

In [ ]:
db[Project].values()

[]

In [ ]:
print(db[Search].select_str)

SELECT "test_db_schema_Search".term AS term,
       "test_db_schema_Search".result_count AS result_count
FROM "test_db_schema_Search"


In [ ]:
print(db[Search.term].select_str)

SELECT "test_db_schema_Search".term AS term
FROM "test_db_schema_Search"


In [ ]:
print(db[Search.result_count].select_str)

SELECT "test_db_schema_Search".term AS term,
       "test_db_schema_Search".result_count AS result_count
FROM "test_db_schema_Search"


In [ ]:
print(db[Search.results].select_str)

SELECT "test_db_schema_Search".term AS term,
       "test_db_schema_Project".number AS number,
       "test_db_schema_Project".status AS status,
       "test_db_schema_Project"."end" AS "end",
       "test_db_schema_Project".start AS
START, "test_db_schema_Project".name AS name
FROM "test_db_schema_Search"
JOIN "test_db_schema_SearchResult" ON "test_db_schema_SearchResult"."test_db_schema_Search_term" = "test_db_schema_Search".term
JOIN "test_db_schema_Project" ON "test_db_schema_SearchResult"."test_db_schema_Project_number" = "test_db_schema_Project".number


In [ ]:
db[Search.term].values()

['test']

In [ ]:
list(db[Search.result_count].items())

[('test', 3)]

In [ ]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         103563 function calls (97694 primitive calls) in 0.253 seconds

   Ordered by: cumulative time
   List reduced from 2069 to 120 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.046    0.023 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:4355(_update_dict)
        1    0.000    0.000    0.034    0.034 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:3523(engine)
        2    0.000    0.000    0.024    0.012 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:830(to_df)
    40/25    0.000    0.000    0.020    0.001 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1694(__get__)
        3    0.000    0.000    0.011    0.004 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:688(_load_records)
       27    0.000    0.000    0.009    0.000 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2337(_has_t